# 02 · Composite & Davison (SPEC-B-002, SPEC-B-010)

Compute midpoint composites and Davison positions from the natal fixtures. These outputs seed the report builder.

In [1]:
from __future__ import annotations

import json
import math
import os
from pathlib import Path
import pandas as pd
import sys

ASTRO_ROOT = Path(os.environ.get('ASTROENGINE_ROOT', '..')).resolve()
DOCS_ROOT = Path(os.environ.get('DOCS_SITE_ROOT', 'docs-site')).resolve()
FIXTURES = DOCS_ROOT / 'docs' / 'fixtures'
os.environ.setdefault('SE_EPHE_PATH', str(ASTRO_ROOT / 'datasets' / 'swisseph_stub'))
if str(ASTRO_ROOT) not in sys.path:
    sys.path.insert(0, str(ASTRO_ROOT))

In [2]:
with open(FIXTURES / 'positions_subject.json') as fh:
    subject = json.load(fh)
with open(FIXTURES / 'positions_partner.json') as fh:
    partner = json.load(fh)

subject_lons = {k: v['longitude'] for k, v in subject.items()}
partner_lons = {k: v['longitude'] for k, v in partner.items()}
list(subject_lons.items())[:3]

[('Jupiter', 111.61446471656818),
 ('Mars', 29.133036444205217),
 ('Mercury', 118.68755174469528)]

In [3]:
events = pd.read_csv(FIXTURES / 'birth_events.csv')
subject_row = events.iloc[0]
partner_row = events.iloc[1]

In [4]:
from astroengine.core.rel_plus.composite import composite_midpoint_positions, davison_positions
from astroengine.chart.natal import ChartLocation, compute_natal_chart
from astroengine.ephemeris.swisseph_adapter import SwissEphemerisAdapter

adapter = SwissEphemerisAdapter()
objects = ['Sun', 'Moon', 'Mercury', 'Venus', 'Mars']
composite = composite_midpoint_positions(subject_lons, partner_lons, objects)
composite

{'Sun': 52.13802041737405,
 'Moon': 51.1181379706091,
 'Mercury': 64.95093300691764,
 'Venus': 25.8503583273406,
 'Mars': 354.82568139459363}

In [5]:
from datetime import datetime

def provider(when: datetime) -> dict[str, float]:
    chart = compute_natal_chart(when, ChartLocation(latitude=0.0, longitude=0.0), adapter=adapter)
    return {name: pos.longitude for name, pos in chart.positions.items()}

davison = davison_positions(objects, pd.to_datetime(subject_row['ts']).to_pydatetime(), pd.to_datetime(partner_row['ts']).to_pydatetime(), provider, lat_a=subject_row['lat'], lon_a=subject_row['lon'], lat_b=partner_row['lat'], lon_b=partner_row['lon'])
davison

{'Sun': 52.803398258066,
 'Moon': 51.37323552107983,
 'Mercury': 26.840129290252843,
 'Venus': 95.60026765291965,
 'Mars': 112.78955318861482}

In [6]:
fixture_comp = json.loads((FIXTURES / 'composite_midpoints.json').read_text())
fixture_davison = json.loads((FIXTURES / 'davison_positions.json').read_text())

print('Composite delta:', max(abs(composite[k] - fixture_comp[k]) for k in objects))
print('Davison delta:', max(abs(davison[k] - fixture_davison[k]) for k in objects))

Composite delta: 0.0
Davison delta: 0.0


In [7]:
import hashlib
print('Results checksum:', hashlib.sha256((FIXTURES / 'composite_midpoints.json').read_bytes()).hexdigest())

Results checksum: 40b07eac981afa58094387af91b444844a967d67f658e9bfcd5e37f6e80e8d5c
